本文详细介绍文本分类任务finetune的流程，以及基础配置和高级配置的使用。
## Train的调用流程
### 载入数据
ModelScope可以提供了标准的`MsDataset`接口供用户进行基于ModelScope生态的数据源加载，也支持来自第三方库用户自定义数据集加载，如NLP领域的`Datasets`库。
具体示例如下，如何加载NLP领域里面的clue榜单的afqmc（Ant Financial Question Matching Corpus）数据集：


In [1]:
# Option 1: 使用ModelScope dataset-hub上存储的原生的MsDataset
from modelscope.msdatasets import MsDataset 
dataset = MsDataset.load('clue', subset_name='afqmc', split='train')

# Option 2: 通过MsDataset加载huggingface上的数据集
from modelscope.msdatasets import MsDataset
from modelscope.utils.constant import Hubs

dataset = MsDataset.load('clue', subset_name='afqmc', split='train', hub = Hubs.huggingface)

# Option 3: 直接使用huggingface的api加载huggingface数据集
# 可参考huggingface/Datasets说明：https://huggingface.co/docs/datasets/index 



具体MsDataset 使用可以参考接口文档： [数据的处理](../../数据集/数据集介绍.ipynb); 

### 数据预处理
相同任务的训练和推理的数据预处理，可以采用相同的Preprocessor。
在配置文件中传入注册的 preprocessor名称即可，trainer在build阶段会自动加载相应preprocessor，并根据当前Mode置于 `traine`或者`eval `的状态。如下面示例代码，在进行NLP情感分类下游任务`sentiment-classification`的finetune过程，需要调用的preprocessor在configuration.json配置与推理阶段相同，会通过type = 'sen-cls-tokenizer' 构建出对应的前处理模块。
同时，在处理dataset过程中如何申明dataset中label的字段、样本名称字段和样本种类等信息，均需要添加到配置文件的的train.dataset字段中。

```json
{
  "task": "sentiment-classification",
  "preprocessor":{
    "type": "sen-cls-tokenizer",
  },
  "model": {
      "type": "structbert",
  },
  "framework": "pytorch",
  "train": {
      "dataset": {
        "train": {
          "labels": ["0", "1"],
          "first_sequence": "sentence1",
          "second_sequence": "sentence2",
          "label": "label",
        }
      }, 
      ...
  },
  "evaluation": {
     ...
  },    
}

```
具体的预处理方法可在对应注册为sen-cls-tokenizer的 前处理模块进行开发，相应使用方法参考 Preprocessor接口文档：[数据的预处理](../../ModelScope%20Library教程/数据的预处理.ipynb)
### 训练
由trainer相关的接口文档可以了解到，训练过程核心流程由dataset、dataloader、optimizer、lr_scheduler和hooks等组件功能组成，具体是通过在configuration.json配置文件中申明的方式注册进入trainer的流程中，具体参考：[configuration详解](../Configuration详解.ipynb)
#### 基础配置
在训练开始前需要配置好相应的trainer配置文件， 下面给一个完整的NLP进行情感分类下游任务finetune的配置。
用户在实际使用过程中，如果示例无法提供帮助，可以根据自己实际训练要求，针对optimizer/lr_scheduler/hooks进行定制注册，并在配置文件中通过type字段申明相应定制方法进行使用。

```json
{
  "task": "sentiment-classification",
  "preprocessor":{
    "type": "sen-cls-tokenizer",
  },
  "model": {
      "type": "structbert",
  },
  "framework": "pytorch",
  "train": {
      "work_dir": "/tmp",
      "max_epochs": 5,
      "dataset": {
          "train": {
              "labels": ["0", "1"],
              "first_sequence": "sentence1",
              "second_sequence": "sentence2",
              "label": "label",
          },
      },
      "dataloader": {
          "batch_size_per_gpu": 32,
          "workers_per_gpu": 1
      },
      "optimizer": {
          "type": "AdamW",
          "lr": 2e-5,
          "options": {}
      },
      "lr_scheduler": {
          "type": "LinearLR",
          "start_factor": 1.0,
          "end_factor": 0.0,
          "total_iters": null,
          "options": { 
              "by_epoch": false
          }
      },
      "hooks": [{
          "type": "CheckpointHook",
          "interval": 1
      }, {
          "type": "TextLoggerHook",
          "interval": 1
      }, {
          "type": "IterTimerHook"
      }, {
          "type": "EvaluationHook",
          "by_epoch": false,
          "interval": 100
      }]
  },
  "evaluation": {
      "dataloader": {
          "batch_size_per_gpu": 32,
          "workers_per_gpu": 1,
          "shuffle": false
      }
  }

}

```

综合上述内容，可通过如下代码进行模型的finetune训练



In [1]:
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer

workspace = "/worksapce"

# 如果数据集在MsDataset中存在，推荐使用MsDataset.load方法
train_dataset = MsDataset.load('afqmc_small', split='train')
eval_dataset = MsDataset.load('afqmc_small', split='validation')

# 也可以读取huggingface hub上存储的数据
# train_dataset = MsDataset.load('afqmc_small', split='train', hub = Hubs.huggingface)
# eval_dataset = MsDataset.load('afqmc_small', split='validation', hub = Hubs.huggingface)

model_id = 'damo/nlp_structbert_sentence-similarity_chinese-base'
kwargs = dict(
    model=model_id,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    work_dir=workspace)

trainer = build_trainer(default_args=kwargs)
trainer.train()


#### 高级配置
在实际过程用户可能会频繁对配置进行调整，不光是训练相关的参数，很可能对应的下游任务都会变化，因此我们提供了高级配置方式供算法用户使用，从而减少不必要的configuration文件改写。
具体包括自定义cfg文件覆盖，以及自定义cfg_function的方法进行。

- 自定义cfg file， 通过覆盖更新cfg_file文件进行代码内的配置调整


In [1]:
import os
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer
from modelscope.utils.hub import read_config

workspace = "/worksapce"

 # 如果数据集在MsDataset中存在，推荐使用MsDataset.load方法
train_dataset = MsDataset.load('afqmc_small', split='train')
eval_dataset = MsDataset.load('afqmc_small', split='validation')

# 也可以读取huggingface hub上存储的数据
# train_dataset = MsDataset.load('afqmc_small', split='train', hub = Hubs.huggingface)
# eval_dataset = MsDataset.load('afqmc_small', split='validation', hub = Hubs.huggingface)

model_id = 'damo/nlp_structbert_sentence-similarity_chinese-base'
cfg = read_config(model_id)
cfg.train.max_epochs = 20
cfg.train.work_dir = tmp_dir
cfg_file = os.path.join(tmp_dir, 'config.json')
cfg.dump(cfg_file)
kwargs = dict(
    model=model_id,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    work_dir=workspace,
    cfg_file=cfg_file,
)

trainer = build_trainer(default_args=kwargs)
trainer.train()



- NLP模型可以使用自定义cfg_function申明，通过代码内自定义相应内容进行配置更新



In [1]:
import os
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer
from modelscope.utils.hub import read_config

workspace = "/worksapce"

 # 如果数据集在MsDataset中存在，推荐使用MsDataset.load方法
train_dataset = MsDataset.load('afqmc_small', split='train')
eval_dataset = MsDataset.load('afqmc_small', split='validation')

def cfg_modify_fn(cfg):
    cfg.task = 'sentence-similarity'
    cfg['preprocessor'] = {'type': 'sen-sim-tokenizer'}
    cfg.train.optimizer.lr = 2e-5
    cfg['dataset'] = {
        'train': {
            'labels': ['0', '1'],
            'first_sequence': 'sentence1',
            'second_sequence': 'sentence2',
            'label': 'label',
        }
    }
    cfg.train.max_epochs = 10
    cfg.train.lr_scheduler = {
        'type': 'LinearLR',
        'start_factor': 1.0,
        'end_factor': 0.0,
        'total_iters':
        int(len(train_dataset) / 32) * cfg.train.max_epochs,
        'options': {
            'by_epoch': False
        }
    }
    cfg.train.hooks = [{
        'type': 'CheckpointHook',
        'interval': 1
    }, {
        'type': 'TextLoggerHook',
        'interval': 1
    }, {
        'type': 'IterTimerHook'
    }, {
        'type': 'EvaluationHook',
        'by_epoch': False,
        'interval': 100
    }]
    return cfg

# 也可以读取huggingface hub上存储的数据
# train_dataset = MsDataset.load('afqmc_small', split='train', hub = Hubs.huggingface)
# eval_dataset = MsDataset.load('afqmc_small', split='validation', hub = Hubs.huggingface)

model_id = 'damo/nlp_structbert_backbone_tiny_std'
kwargs = dict(
    model=model_id,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    work_dir=workspace,
    cfg_modify_fn=cfg_modify_fn,
)

trainer = build_trainer(name='nlp-base-trainer', default_args=kwargs)
trainer.train()


### 评估
#### 交叉验证
交叉验证是在train时同步进行的，基于在配置文件中的 train.hooks的 EvaluationHook，具体配置如下：


In [1]:
{
   ...
  "train": {
     ...
      "hooks": [
          ...
          , {
          "type": "EvaluationHook",
          "by_epoch": false,
          "interval": 100
      }]
  },

}


用户可以根据自己实际情况进行调整，也可自行注册相应hook，并通过type字段注册在配置文件中进行调用。
#### 训练后验证

1. 指定并加载验证数据集
2. build_trainer
3. 调用evaluate方法

如下代码展示了模型的验证流程


In [1]:
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer

# 如果数据集在MsDataset中存在，推荐使用MsDataset.load方法
train_dataset = MsDataset.load('afqmc_small', split='train')
eval_dataset = MsDataset.load('afqmc_small', split='validation')

# 也可以读取huggingface hub上存储的数据
# train_dataset = MsDataset.load('afqmc_small', split='train', hub = Hubs.huggingface)
# eval_dataset = MsDataset.load('afqmc_small', split='validation', hub = Hubs.huggingface)

kwargs = dict(
    model='damo/nlp_structbert_sentence-similarity_chinese-base',
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    work_dir='/workspace')

trainer = build_trainer(default_args=kwargs)
# 直接调用trainer.evaluate，可以传入train阶段生成的ckpt
# 也可以不传入参数，直接验证model
metrics = trainer.evaluate(checkpoint_path=None)
print(metrics)



### 指标
指标用来衡量某个具体任务的验证结果，用户可以查看如下package找到已支持指标类的列表：


In [1]:
import modelscope.metrics


以NLP领域举例，目前支持的指标有以下几种：

- SequenceClassificationMetric
   - 指标名称为seq-cls-metric
   - 为分类任务提供，返回的指标为accuracy
- TokenClassificationMetric
   - 指标名称为token-cls-metric
   - 为序列标注任务提供，返回的指标为F1/recall/precision/accuracy
   - 支持提供每个子类别的指标，通过return_entity_level_metrics=True开启
- TextGenerationMetric
   - 指标名称为text-gen-metric
   - 为生成任务提供，返回的指标为根据rouge计算的F1

代码中我们为支持finetune的各任务类型指定了默认metric类，：


In [1]:
from modelscope.metrics.builder import task_default_metrics


用户也可以在cfg中指定自己需要的指标类型：


In [1]:
# 指定使用SequenceClassificationMetric
cfg.evaluation.metrics = 'seq-cls-metric'


指标的具体介绍可以参考：[模型的评估](../../ModelScope%20Library教程/模型的评估.ipynb)
